# step2: RetrievalQA 체인 구현

1. 2주차에서 만든 Document Pipeline 재사용
2. Retriever 설정
3. RetrievalQA 체인 구현 (LCEL 스타일)
4. 기술 문서 Q&A 챗봇 인터페이스


In [ ]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

from src.document_pipeline import DocumentPipeline

load_dotenv()


## 설정 파라미터

검색 및 체인 파라미터를 상단에서 설정합니다.


In [ ]:
# Retriever 설정
SEARCH_TYPE = "similarity"
TOP_K = 3

# 벡터 저장소 설정
VECTOR_STORE_TYPE = "chroma"
CHROMA_PERSIST_DIR = "./chroma_db"
FAISS_SAVE_DIR = "./faiss_db"

print(f"검색 타입: {SEARCH_TYPE}")
print(f"Top K: {TOP_K}")
print(f"벡터 저장소 타입: {VECTOR_STORE_TYPE}")


## 1. 벡터 저장소 로드

2주차에서 생성한 벡터 저장소를 로드합니다.
ㅝ

In [ ]:
pipeline = DocumentPipeline()

if VECTOR_STORE_TYPE == "chroma":
    vectorstore = pipeline.load_chroma_store(CHROMA_PERSIST_DIR)
    print(f"Chroma 벡터 저장소 로드 완료: {CHROMA_PERSIST_DIR}")
elif VECTOR_STORE_TYPE == "faiss":
    vectorstore = pipeline.load_faiss_store(FAISS_SAVE_DIR)
    print(f"FAISS 벡터 저장소 로드 완료: {FAISS_SAVE_DIR}")
else:
    raise ValueError(f"지원하지 않는 벡터 저장소 타입: {VECTOR_STORE_TYPE}")

print(f"벡터 저장소 타입: {type(vectorstore).__name__}")


## 2. Retriever 설정

벡터 저장소로부터 retriever 객체를 생성합니다.



In [ ]:
retriever = vectorstore.as_retriever(
    search_type=SEARCH_TYPE,
    search_kwargs={"k": TOP_K}
)

print(f"Retriever 생성 완료")
print(f"  - 검색 타입: {SEARCH_TYPE}")
print(f"  - Top K: {TOP_K}")

# 테스트 검색
test_query = "Tesla revenue"
test_docs = retriever.invoke(test_query)
print(f"\n테스트 검색 ('{test_query}') 결과: {len(test_docs)}개 문서")


## 3. LLM 초기화


In [ ]:
gpt_api_key = os.getenv("GPT_API_KEY")
llm = None

if gpt_api_key:
    llm = ChatOpenAI(
        model="gpt-4o-mini",
        api_key=gpt_api_key,
        temperature=0
    )
    print("GPT LLM 초기화 완료")
else:
    print("GPT_API_KEY가 없어서 LLM을 사용할 수 없습니다.")
    print("환경 변수를 설정해주세요.")


## 4. 프롬프트 설계

역할/태도, 근거 사용, 출력 형식을 포함한 프롬프트를 설계합니다.


In [ ]:
def format_docs(docs):
    """검색된 문서를 포맷팅합니다."""
    formatted_parts = []
    sources = []
    
    for i, doc in enumerate(docs, 1):
        doc_type = doc.metadata.get('type', 'text')
        source = doc.metadata.get('source', 'N/A')
        page = doc.metadata.get('page', 'N/A')
        
        if doc_type == 'table':
            caption = doc.metadata.get('caption', 'N/A')
            original_table = doc.metadata.get('original_table', doc.page_content)
            formatted_parts.append(f"[문서 {i} - 표: {caption}]\n{original_table}")
            sources.append({
                'index': i,
                'type': 'table',
                'caption': caption,
                'source': source,
                'page': page
            })
        else:
            formatted_parts.append(f"[문서 {i}]\n{doc.page_content}")
            sources.append({
                'index': i,
                'type': 'text',
                'source': source,
                'page': page
            })
    
    return "\n\n".join(formatted_parts), sources

template = """당신은 주어진 기술 문서에 기반해서만 답변하는 AI 어시스턴트입니다.

다음 문서들을 참고하여 질문에 답변해주세요. 반드시 제공된 문서 내용을 기반으로 답변하고, 문서에 없는 내용은 '문서에 정보가 없습니다'라고 답하세요.

문서:
{context}

질문: {question}

답변 형식:
1. 1줄 요약 답변
2. 상세 답변
3. 근거가 된 문서 정보 (문서 번호, 출처, 페이지/섹션)

답변:"""

prompt = ChatPromptTemplate.from_template(template)


## 5. RetrievalQA 체인 구현 (LCEL 스타일)

retriever | prompt | llm 형태로 파이프라인을 구성합니다.


In [ ]:
def create_rag_chain(retriever, prompt, llm):
    """RAG 체인을 생성합니다 (LCEL 스타일)."""
    def format_context(docs):
        formatted_text, _ = format_docs(docs)
        return formatted_text
    
    chain = (
        {"context": retriever | format_context, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )
    
    return chain

if llm:
    rag_chain = create_rag_chain(retriever, prompt, llm)
    print("RetrievalQA 체인 생성 완료 (LCEL 스타일)")
    print("체인 구조: retriever | format_context | prompt | llm | StrOutputParser")
else:
    print("LLM이 없어서 체인을 생성할 수 없습니다.")


## 6. 체인 테스트


In [ ]:
if llm:
    test_question = "What is Tesla's revenue in Q3 2025?"
    
    print("="*60)
    print("질문:", test_question)
    print("="*60)
    
    # 검색된 문서 확인
    retrieved_docs = retriever.invoke(test_question)
    print(f"\n검색된 문서 수: {len(retrieved_docs)}")
    for i, doc in enumerate(retrieved_docs, 1):
        doc_type = doc.metadata.get('type', 'text')
        source = doc.metadata.get('source', 'N/A')
        page = doc.metadata.get('page', 'N/A')
        if doc_type == 'table':
            caption = doc.metadata.get('caption', 'N/A')
            print(f"  [{i}] 표: {caption} (출처: {source}, 페이지: {page})")
        else:
            print(f"  [{i}] 텍스트 (출처: {source}, 페이지: {page})")
    
    print("\n" + "="*60)
    print("답변:")
    print("="*60)
    
    answer = rag_chain.invoke(test_question)
    print(answer)
else:
    print("LLM이 없어서 테스트를 수행할 수 없습니다.")


## 7. 기술 문서 Q&A 챗봇 인터페이스

콘솔 기반 인터페이스를 구현합니다.



In [ ]:
def chat_interface(rag_chain, retriever):
    """Q&A 챗봇 인터페이스"""
    print("="*60)
    print("기술 문서 Q&A 챗봇")
    print("="*60)
    print("질문을 입력하세요. 종료하려면 'quit', 'exit', 'q'를 입력하세요.\n")
    
    while True:
        question = input("질문: ").strip()
        
        if question.lower() in ['quit', 'exit', 'q', '']:
            print("챗봇을 종료합니다.")
            break
        
        if not question:
            continue
        
        print("\n" + "-"*60)
        print("검색 중...")
        
        try:
            retrieved_docs = retriever.invoke(question)
            print(f"검색된 문서: {len(retrieved_docs)}개\n")
            
            print("답변 생성 중...\n")
            answer = rag_chain.invoke(question)
            
            print("="*60)
            print("답변:")
            print("="*60)
            print(answer)
            
            print("\n" + "-"*60)
            print("참고 문서:")
            for i, doc in enumerate(retrieved_docs, 1):
                doc_type = doc.metadata.get('type', 'text')
                source = doc.metadata.get('source', 'N/A')
                page = doc.metadata.get('page', 'N/A')
                if doc_type == 'table':
                    caption = doc.metadata.get('caption', 'N/A')
                    print(f"  [{i}] 표: {caption}")
                    print(f"      출처: {source} (페이지: {page})")
                else:
                    print(f"  [{i}] 텍스트")
                    print(f"      출처: {source} (페이지: {page})")
                    print(f"      내용 미리보기: {doc.page_content[:100]}...")
            
            print("\n" + "="*60 + "\n")
            
        except Exception as e:
            print(f"오류 발생: {e}")
            print("\n")

if llm:
    print("챗봇 인터페이스를 시작하려면 아래 셀을 실행하세요.")
    print("chat_interface(rag_chain, retriever)")
else:
    print("LLM이 없어서 챗봇 인터페이스를 사용할 수 없습니다.")


In [ ]:
# 챗봇 인터페이스 실행
if llm:
    chat_interface(rag_chain, retriever)
else:
    print("LLM이 없어서 챗봇을 실행할 수 없습니다.")
